In [8]:
# Importing Required library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# for intractivity
from ipywidgets import interact

In [9]:
import sklearn
print(sklearn.__version__) 

1.0.2


In [10]:
# Loading dataset
df = pd.read_csv("incident.csv")
df.head()

,Unnamed: 0,Product,Business object,Issue,Solution
0,0,C4C,SalesQuote,submit disabled,2629811 - Action Submit is Disabled For Sales ...
1,1,C4C,SalesQuote,price,2940435 - Product Price Missing From Sales Quote
2,2,C4C,SalesQuote,copy,3076907 - Can't Copy CPQ Quotes
3,3,C4C,SalesQuote,external indicator,2611062 - How Extern Indicator Works in Sales ...
4,4,C4C,SalesQuote,Scoping,2758035 - Scoping Questions - Sales Quotes


In [11]:
df['Issue'].values

array(['submit disabled', 'price', 'copy', 'external indicator',
       'Scoping ', 'Preview external quote', 'BTD Reference',
       'New button grayed out', 'status completed', 'Reapproval ',
       'Inbound replication', 'Replication status reset', 'delete quote',
       'XML Payload', 'Pricing facet', 'Delete CPQ', 'offline', 'pricing',
       'Replication S4', 'Item party replication', 'AFS Integration',
       'Partner address ', 'Request date', 'Approval status',
       'Delivery priority', 'edit sales order', 'maintainance mode',
       'offline determination', 'delivery block', 'custom status',
       'Open opportunity from account', 'sales phase', 'replication',
       'trend analytics', 'import status', 'ATP check', 'delete ticket',
       'attachments Odata', 'reply on ticket', 'outbound message',
       'non-active employees', 'attachment replication', 'email bounce',
       'email address sync', 'outbound size ', 'create ticket',
       'debug email', 'incoming email', 'c

In [12]:
df.rename(columns={'Solution ':'Solution'},inplace=True)

In [13]:
df=df[['Product','Business object','Issue','Solution']]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Product          49 non-null     object
 1   Business object  49 non-null     object
 2   Issue            49 non-null     object
 3   Solution         49 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


In [15]:
df.describe(include='all')

,Product,Business object,Issue,Solution
count,49,49,49,49
unique,1,5,49,49
top,C4C,SalesQuote,ATP check,3080707 - Cannot Change Sales Phase Field in a...
freq,49,16,1,1


### Extracting Training Data

In [16]:
X=df[['Product','Business object','Issue']]
y=df['Solution']

In [17]:
y.shape

(49,)

In [18]:
X.shape

(49, 3)

### Applying train and test split

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline



In [22]:
# creating onehot encoder
ohe=OneHotEncoder()
ohe.fit(X[['Product','Business object','Issue']])


OneHotEncoder()

In [20]:
# Creating a column transformer to transform categorical columns


In [23]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Product','Business object','Issue']),
                                    remainder='passthrough')

In [24]:
# Linear regression model
lr= LogisticRegression()

In [25]:
pipe=make_pipeline(column_trans,lr)

In [26]:
# fitting the model
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['C4C'], dtype=object),
                                                                            array(['EmailChannel', 'Opportunity', 'SalesOrder', 'SalesQuote',
       'tickets'], dtype=object),
                                                                            array(['AFS Integration', 'ATP check', 'Approval status', 'BTD Reference',
       'Delete CPQ', 'Delivery priority', '...
       'import status', 'incoming email', 'maintainance mode',
       'non-active employees', 'offline', 'offline determination',
       'outbound message', 'outbound size ', 'price', 'pricing',
       'replication', 'reply on ticket', 'sales phase',
       'status completed', 'submit disabled', 'trend analytics'],
      dtype=object)]),
               

In [27]:
y_pred=pipe.predict(X_test)

In [28]:
pipe.predict(pd.DataFrame(columns=X_test.columns,data=np.array(['C4C','SalesOrder','status completed']).reshape(1,3)))

array(['2720090 - Sales Quote Status Does Not Change to Completed'],
      dtype=object)

In [29]:
import pickle

In [30]:
pickle.dump(pipe,open('lr.pkl','wb'))

In [31]:
pipe.predict(pd.DataFrame([['C4C','SalesOrder','reply on ticket']],
                         columns=['Product','Business object','Issue']))

array(['2586440 - Outlook signature not being added to replies for Ticket'],
      dtype=object)

In [36]:
prediction=pipe.predict(pd.DataFrame(columns=['Product','Business object','Issue'],
                              data=np.array(['C4C','SalesOrder','reply on ticket']).reshape(1, 3)))
prediction

array(['2586440 - Outlook signature not being added to replies for Ticket'],
      dtype=object)

In [33]:
pipe.steps[0][1].transformers[0][1].categories[0]

array(['C4C'], dtype=object)